In [ ]:
# Set of services to be scheduled, including dependencies, computational requirement,
# memory requirement and number of containers
services = {
    "1":{"prev": [], "connections":["2"], "cal": 7, "str": 10, "container": 5},
    "2":{"prev": ["1"], "connections":["4"], "cal": 6, "str": 12, "container": 3},
    "3":{"prev": [], "connections":["4"], "cal": 8, "str": 16, "container": 4},
    "4":{"prev": ["2", "3"], "connections":["5"], "cal": 6, "str": 10, "container": 3},
    "5":{"prev": ["4"],"connections":["6", "7"],"cal": 5, "str": 14, "container": 5},
    "6":{"prev": ["5"], "connections":["7"], "cal": 7, "str": 12, "container": 4},
    "7":{"prev": ["5", "6"], "connections":["8"], "cal": 6, "str": 10, "container": 3},
    "8":{"prev": ["6", "7"], "connections":[], "cal": 8, "str": 16, "container": 5}
}

# Set of nodes to schedule to, including computational capacity, memory capacity,
# and fail rate
nodes = {
    "1":{"max_cal": 70, "max_str": 100, "fail": 0.4},
    "2":{"max_cal": 60, "max_str": 110, "fail": 0.01},
    "3":{"max_cal": 80, "max_str": 90, "fail": 0.03},
    "4":{"max_cal": 90, "max_str": 120, "fail": 0.02},
    "5":{"max_cal": 70, "max_str": 80, "fail": 0.05},
    "6":{"max_cal": 90, "max_str": 100, "fail": 0.04},
    "7":{"max_cal": 50, "max_str": 110, "fail": 0.02},
    "8":{"max_cal": 80, "max_str": 120, "fail": 0.06},
    "9":{"max_cal": 60, "max_str": 130, "fail": 0.03},
    "10":{"max_cal": 50, "max_str": 100, "fail": 0.02},
}


In [ ]:
import numpy as np

# Initialize pheromone matrix
pheromones = np.ones((len(services), len(nodes))) * 0.1

# Turn service set into list
added = set()
service_list = []
while len(service_list) < len(services.items()):
    for service in services.keys():
        service_name = str(service)
        prev = set(services[service]["prev"])
        if service_name not in added and prev.issubset(added):
            service_list.append(service_name)
    added.update(service_list)

print(service_list)

#Turn nodes set into list
node_list = [str(node) for node in nodes.keys()]

print(node_list)

['1', '3', '2', '4', '5', '6', '7', '8']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


In [ ]:
# A list of assigned containers for each node
assign_list = [[] for node in node_list]

# Keep track of how many available resources there are
avail = [(nodes[str(node)]["max_cal"], nodes[str(node)]["max_str"]) for node in node_list]

#For each service
for service in service_list:

  #Records the remaining computational and storage capacity
  remaining = np.zeros(len(node_list))

  #For each container in the service
  for container in range(services[service]["container"]):

    #Sums total resources
    total_avail = np.array([x + y for x, y in avail])

    #Get probailities based off percentage of total resources in cluster
    probabilities = total_avail/np.sum(total_avail)

    #Get the node with the most resources
    node = np.argmax(probabilities)
    r_cal, r_str = avail[node]

    #If can't schedule to that node, choose random node until it finds an acceptable node
    while r_cal <= services[service]["cal"] or r_str <= services[service]["str"]:
      node = int(np.random.choice(node_list, p = probabilities)) - 1
      r_cal, r_str = avail[node]

    #Assign the service to the specific node
    assign_list[node].append(service)

    #Update available resources for that node
    avail[node] = tuple(np.subtract(avail[node], (services[service]["cal"], services[service]["str"])))

#Store path and remaining resources avaialble
path = [(index + 1, assign_list[index])for index in range(len(assign_list))]
remaining = [(index + 1, avail[index])for index in range(len(avail))]

[(1, ['2', '4', '7']), (2, ['2', '5', '7']), (3, ['2', '5', '7']), (4, ['1', '1', '3', '5', '8']), (5, ['5', '8']), (6, ['1', '3', '6', '8']), (7, ['4', '6']), (8, ['1', '3', '4', '6']), (9, ['1', '3', '6', '8']), (10, ['5', '8'])]
[(1, (52, 68)), (2, (43, 74)), (3, (63, 54)), (4, (55, 54)), (5, (57, 50)), (6, (60, 46)), (7, (37, 88)), (8, (52, 72)), (9, (30, 76)), (10, (37, 70))]


In [ ]:
#Print the service schedule
print(f'\n1. Microservices Assigned\n')
print(f'Node \t\tScheduled')
print(f'----------------------------------------------')
for i in path:
  print(f'Node {i[0]}: \t{i[1]} ')


#Print the remaining resouces of each node
print(f'\n\n2. Remaining Resources Summary\n')
print(f'Node \t\tCPU Usage \tMemory Usage')
print(f'----------------------------------------------')
for i, val in remaining:
  print(f'Node {i}: \tcpu:{val[0]}/{list(nodes[str(i)].values())[0]} () \tmem:{val[1]}/{list(nodes[str(i)].values())[1]}')


1. Microservices Assigned

Node 		Scheduled
----------------------------------------------
Node 1: 	['2', '4', '7'] 
Node 2: 	['2', '5', '7'] 
Node 3: 	['2', '5', '7'] 
Node 4: 	['1', '1', '3', '5', '8'] 
Node 5: 	['5', '8'] 
Node 6: 	['1', '3', '6', '8'] 
Node 7: 	['4', '6'] 
Node 8: 	['1', '3', '4', '6'] 
Node 9: 	['1', '3', '6', '8'] 
Node 10: 	['5', '8'] 


2. Remaining Resources Summary

Node 		CPU Usage 	Memory Usage
----------------------------------------------
Node 1: 	cpu:52/70 () 	mem:68/100
Node 2: 	cpu:43/60 () 	mem:74/110
Node 3: 	cpu:63/80 () 	mem:54/90
Node 4: 	cpu:55/90 () 	mem:54/120
Node 5: 	cpu:57/70 () 	mem:50/80
Node 6: 	cpu:60/90 () 	mem:46/100
Node 7: 	cpu:37/50 () 	mem:88/110
Node 8: 	cpu:52/80 () 	mem:72/120
Node 9: 	cpu:30/60 () 	mem:76/130
Node 10: 	cpu:37/50 () 	mem:70/100


In [ ]:
#Calculate utilization rate from best computed schedule
utilization_rate = [(node, 1 - (cal + mem)/(nodes[str(node)]["max_cal"] + nodes[str(node)]["max_str"])) for (node, (cal, mem)) in remaining]

#Print utilization rate for each node from best computed schedule.
print(f'Total Utilization Rate for each Node\n')
print(f'Node \t\tUtilization')
print(f'-----------------------------------')
for i, util in utilization_rate:
  print(f'Node {i}: \t{util}')

Total Utilization Rate for each Node

Node 		Utilization
-----------------------------------
Node 1: 	0.2941176470588235
Node 2: 	0.31176470588235294
Node 3: 	0.31176470588235294
Node 4: 	0.4809523809523809
Node 5: 	0.2866666666666666
Node 6: 	0.4421052631578948
Node 7: 	0.21875
Node 8: 	0.38
Node 9: 	0.4421052631578948
Node 10: 	0.2866666666666666


In [ ]:
#Print the average utilization of the nodes
total_utilization = 0
for _, util in utilization_rate:
  total_utilization += util/10
print(total_utilization)

0.34548932994250336
